In [86]:
import pandas as pd
import numpy as np
import csv
import gmaps
import requests
import json

from config import gkey
gmaps.configure(api_key=gkey)

In [87]:
# Import the population.csv file as a DataFrame
state_df = pd.read_csv("resources/state_populations_2000_to_2019.csv")
county_df = pd.read_csv("resources/county_populations_2000_to_2019.csv")


pop_data = pd.merge(county_df,state_df,
                     on="state_name",
                     how="left")
# displaying result
print(pop_data)

     state_name        county_name  county_FIPS  pop_2000_x  pop_2001_x  \
0       Alabama     Autauga County         1001       43872       44434   
1       Alabama     Baldwin County         1003      141358      144988   
2       Alabama     Barbour County         1005       29035       29223   
3       Alabama        Bibb County         1007       19936       20942   
4       Alabama      Blount County         1009       51181       51999   
...         ...                ...          ...         ...         ...   
3138    Wyoming  Sweetwater County        56037       37484       36566   
3139    Wyoming       Teton County        56039       18367       18594   
3140    Wyoming       Uinta County        56041       19662       19406   
3141    Wyoming    Washakie County        56043        8257        8023   
3142    Wyoming      Weston County        56045        6635        6472   

      pop_2002_x  pop_2003_x  pop_2004_x  pop_2005_x  pop_2006_x  ...  \
0          45157       457

In [88]:
pop_data.columns

Index(['state_name', 'county_name', 'county_FIPS', 'pop_2000_x', 'pop_2001_x',
       'pop_2002_x', 'pop_2003_x', 'pop_2004_x', 'pop_2005_x', 'pop_2006_x',
       'pop_2007_x', 'pop_2008_x', 'pop_2009_x', 'pop_2010_x', 'pop_2011_x',
       'pop_2012_x', 'pop_2013_x', 'pop_2014_x', 'pop_2015_x', 'pop_2016_x',
       'pop_2017_x', 'pop_2018_x', 'pop_2019_x', 'state_FIPS', 'pop_2000_y',
       'pop_2001_y', 'pop_2002_y', 'pop_2003_y', 'pop_2004_y', 'pop_2005_y',
       'pop_2006_y', 'pop_2007_y', 'pop_2008_y', 'pop_2009_y', 'pop_2010_y',
       'pop_2011_y', 'pop_2012_y', 'pop_2013_y', 'pop_2014_y', 'pop_2015_y',
       'pop_2016_y', 'pop_2017_y', 'pop_2018_y', 'pop_2019_y'],
      dtype='object')

In [89]:
pop_data["county_name"].nunique()

1877

In [90]:
county_df.columns

Index(['state_name', 'county_name', 'county_FIPS', 'pop_2000', 'pop_2001',
       'pop_2002', 'pop_2003', 'pop_2004', 'pop_2005', 'pop_2006', 'pop_2007',
       'pop_2008', 'pop_2009', 'pop_2010', 'pop_2011', 'pop_2012', 'pop_2013',
       'pop_2014', 'pop_2015', 'pop_2016', 'pop_2017', 'pop_2018', 'pop_2019'],
      dtype='object')

In [91]:
state_df.columns

Index(['state_name', 'state_FIPS', 'pop_2000', 'pop_2001', 'pop_2002',
       'pop_2003', 'pop_2004', 'pop_2005', 'pop_2006', 'pop_2007', 'pop_2008',
       'pop_2009', 'pop_2010', 'pop_2011', 'pop_2012', 'pop_2013', 'pop_2014',
       'pop_2015', 'pop_2016', 'pop_2017', 'pop_2018', 'pop_2019'],
      dtype='object')

In [92]:
state_county_df = pop_data[["state_name", "county_name"]]
#sc = state_county_df.groupby (["state_name","county_name"])
state_county_df.nunique()

state_name       51
county_name    1877
dtype: int64

In [93]:
landfill_df = pd.read_csv("resources/openlandfills.csv")
landfill_df.dropna(inplace = True)
landfill_df.head()

,State,Landfill Name,Current Landfill Status,Latitude,Longitude
0,AK,Anchorage Regional Landfill,Open,61.293281,-149.602138
1,AK,Capitol Disposal Landfill,Open,58.352800,-134.494700
2,AK,Central Peninsula Landfill (CPL),Open,60.447140,-151.103690
3,AK,Palmer Central Landfill,Open,61.590000,-149.210000
4,AK,South Cushman Landfill,Open,64.804760,-147.700850


In [94]:
# Store latitude and longitude in locations
locations = landfill_df[["Latitude", "Longitude"]]

In [95]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, 
                                 dissipating=False, max_intensity=8,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
In [ ]:
# load csv
# Combime files
# delete unused columns
# identify state, county
# Merge landfill data, group by state and county
# Identify the highest landfill states
# merge cancer data, overlay on state and conty data
# create a 100 miles radius based on address of landfill
# find similar population and landfill dataa( similar to weather or gmaps homework assignment?)
# If silimair popolations with landfills have similar cancer rates, this may prove a higher risk
# Can we use gmaps to rank the landfills with the highest cancer radius?